In [23]:
import joblib
model_pretrained = joblib.load('house_prices_predictio.pkl')
import pandas as pd
pd.set_option('display.max_rows', 500)

## 將模型導入後，把test.csv的資料做跟當初train.csv一樣的處理(impute採用train data)

In [24]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv("data/test.csv")
df_test = df_test.drop(['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence',
       'MiscFeature'], axis=1)

In [25]:
df_only_num = df_test.select_dtypes(include=['int64','float64']).columns
cat_var=df_test.select_dtypes(include=['O']).columns

In [26]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
imputer.fit(df_train[df_only_num])
df_test[df_only_num]=imputer.transform(df_test[df_only_num])

In [27]:
cat_imputer=SimpleImputer(strategy='most_frequent')
cat_imputer.fit(df_train[cat_var])
df_test[cat_var]=cat_imputer.transform(df_test[cat_var])

In [28]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for var in cat_var:
    le.fit(df_train[var])
    df_test[var]=le.transform(df_test[var])

## 開始預測

In [29]:
predictions = model_pretrained.predict(df_test)

In [30]:
forSubmissionDF = pd.DataFrame(columns=['Id', 'SalePrice'])
forSubmissionDF['Id'] = range(1461, 2920)
forSubmissionDF['SalePrice'] = predictions
forSubmissionDF

Id      SalePrice
0     1461  117985.029051
1     1462  118993.338916
2     1463  176857.808232
3     1464  185911.793657
4     1465  177795.874748
...    ...            ...
1454  2915   71848.935384
1455  2916   67160.182257
1456  2917  164651.005622
1457  2918  116204.187898
1458  2919  235869.051555

[1459 rows x 2 columns]

## 結果輸出

In [31]:
forSubmissionDF.to_csv('for_submission.csv', index=False)

![kaggle](/kaggle.png)

<h2>比賽結果: 3223/4324  RMSE: 0.17939</h2>
<p>推測還可以改善的部分:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;可能可以使用多種模型來堆疊以達到更高的準確程度，或是引用知名模型來做fine tune。資料集的處理也可以更加細膩(分析是否有無關的資料，取出以降低對模型的影響)</p>
<p>不同的嘗試與結果分析:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;我其實有嘗試過要找出對於模型建構來說比較重要的資料，但是就結果來說我認為我對於看圖表的比較是非常的不擅長，常常分分辨不出來該資料集對於模型有沒有幫助。做出來的結果也都沒有這次的成績好，所以沒有使用。</p>